To work with Amazon SageMaker Lakehouse and the Apache Iceberg API in Python, you can leverage the flexibility of querying and managing data across data lakes and warehouses. Below is a concise guide to help you get started:

1. Install Required Libraries
Ensure you have the necessary Python libraries installed:

In [77]:
!pip install boto3 pyiceberg pandas mypy-boto3-glue pyarrow awswrangler

1019569.45s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2. Set Up AWS SDK (boto3)
Use boto3 to interact with AWS services like SageMaker and S3:

In [55]:
import boto3

REGION = "ap-northeast-1"

session = boto3.Session(
    profile_name="default",
    region_name=REGION
)

# Initialize a SageMaker client
sagemaker_client = session.client('sagemaker', region_name=REGION)

# Initialize an S3 client for data storage
s3_client = session.client('s3', region_name=REGION)

3. Query Data Using Apache Iceberg API
Apache Iceberg allows querying data directly from your data lake. Here's an example:

In [45]:
from pyiceberg.catalog.glue import GlueCatalog  

GLUE_REST_API_ENDPOINT = "https://glue.ap-northeast-1.amazonaws.com/iceberg"
DATABASE = "glue_db_bsyeydfwn7q7bt"
TABLE = "mylakehouse_dev_traffic_data"

# Load Iceberg catalog (e.g., AWS Glue or REST-based catalog)
catalog = GlueCatalog(region_name=REGION, name="任意の名前")

# list table in database
print(catalog.list_tables(namespace=DATABASE))

# Access a specific table
table = catalog.load_table(f"{DATABASE}.{TABLE}")


# Query data (e.g., filter rows)
print(table.scan().to_pandas())

[('glue_db_bsyeydfwn7q7bt', 'mylakehouse_dev_traffic_data')]
                                       id           timestamp  age  gender  \
0    4a95add3-991a-421c-ab57-3edd1bb20542 2025-08-16 10:00:02   15  female   
1    012569a0-7841-4ee2-913d-e35d94c225c3 2025-08-16 10:00:03   21  female   
2    a712cd8c-c1c7-46f0-8c72-7f9939363618 2025-08-16 10:00:03   23  female   
3    e3cb3297-54db-421a-929b-a2321549e2a9 2025-08-16 10:00:04   14  female   
4    f2a9127d-2a04-4f88-b37f-897b3fc6791b 2025-08-16 10:00:05   13  female   
..                                    ...                 ...  ...     ...   
205  5b3f970a-4a30-4674-b482-e37088ef2d6d 2025-08-16 10:00:55   13    male   
206  1dcd9f34-e381-41ec-99a7-745980e503e0 2025-08-16 10:00:58   30    male   
207  d943bb27-650d-42e1-b43f-3191e0040b4b 2025-08-16 10:00:59   12    male   
208  dc75e67a-11b1-4c12-8fb7-62b299640e6e 2025-08-16 10:00:03    9    male   
209  26419201-20d1-4e96-9944-c8e62f0ac276 2025-08-16 10:00:15    8    male   

  

4.1 Query Data in Iceberg table Using Athena

In [80]:
import awswrangler as wr
REGION = "ap-northeast-1"
DATASOUCE_FOR_ICEBERGE = "AwsDataCatalog"
GLUE_DB_FOR_ICEBERGE = "glue_db_bsyeydfwn7q7bt"
GLUE_TABLE_FOR_ICEBERGE = "mylakehouse_dev_traffic_data"

S3_OUTPUT_LOCATION = "s3://lakehouse-sample-s3-bucket/athena/"


session = boto3.Session(
    profile_name="default",
    region_name=REGION
)

sql_query = f"""
SELECT t.* from "{DATASOUCE_FOR_ICEBERGE}"."{GLUE_DB_FOR_ICEBERGE}"."{GLUE_TABLE_FOR_ICEBERGE}" AS t limit 10;
"""

df = wr.athena.read_sql_query(sql=sql_query,
                              data_source=DATASOUCE_FOR_ICEBERGE,
                              database=GLUE_DB_FOR_ICEBERGE,
                              s3_output=S3_OUTPUT_LOCATION,
                              ctas_approach=False,
                              boto3_session=session,
)

print(df)


                                     id           timestamp  age  gender  \
0  09ddc699-14e2-469b-93ea-688f7f44abb7 2025-08-31 17:50:00   12  female   
1  f8bc1dfb-3ebc-40aa-8ad1-08ead80639cc 2025-08-31 17:50:00   15    male   
2  62b8c25b-2568-472d-91ce-f40e78484196 2025-08-31 17:50:00   19  female   
3  aeb03501-bc44-4f18-9f2b-c47b7f95b171 2025-08-31 17:50:00   13  female   
4  adb1fccc-b3cd-401c-b099-a7b77943735d 2025-08-31 17:50:01   29    male   
5  351c07dc-1ac6-4bcb-b79f-c71bd9b85d77 2025-08-31 17:50:01   17    male   
6  429c2a3c-36bc-4f61-8969-27f006b0f307 2025-08-31 17:50:01   14  female   
7  c26bee49-377c-4877-a68a-ce5d6ab77009 2025-08-31 17:50:01   18  female   
8  c6e76fea-18d4-4ea5-a1bc-57adae6c09b3 2025-08-31 17:50:01   24  female   
9  ce4b10fc-24a3-4706-ae52-97c0ca4f16e7 2025-08-31 17:50:02   17    male   

   direction  shop_id  
0         -1  0123456  
1         -1  0123456  
2          1  0123456  
3         -1  0123456  
4         -1  0123456  
5          1  01234

4.2 Federated Query Data in Aurora MySQL Using Athena

In [81]:
import awswrangler as wr
REGION = "ap-northeast-1"
ATHENA_DATA_SOURCE_NAME = "lakehouse_sample_mysql_2"
GLUE_DB_FOR_AURORA = "sample_db"
GLUE_TABLE_FOR_AURORA = "shop"

S3_OUTPUT_LOCATION = "s3://lakehouse-sample-s3-bucket/athena/"


session = boto3.Session(
    profile_name="default",
    region_name=REGION
)


sql_query = f"""
SELECT s.* from "{ATHENA_DATA_SOURCE_NAME}"."{GLUE_DB_FOR_AURORA}"."{GLUE_TABLE_FOR_AURORA}" AS s;
"""

df = wr.athena.read_sql_query(sql=sql_query,
                              data_source=ATHENA_DATA_SOURCE_NAME,
                              database=GLUE_DB_FOR_AURORA,
                              s3_output=S3_OUTPUT_LOCATION,
                              ctas_approach=False,
                              boto3_session=session,
)

print(df)


        id name partition_name
0  0123456  店舗1              *


4.3 Join two datasource

In [82]:
import awswrangler as wr
REGION = "ap-northeast-1"

DATASOUCE_FOR_ICEBERGE = "AwsDataCatalog"
GLUE_DB_FOR_ICEBERGE = "glue_db_bsyeydfwn7q7bt"
GLUE_TABLE_FOR_ICEBERGE = "mylakehouse_dev_traffic_data"

ATHENA_DATA_SOURCE_NAME = "lakehouse_sample_mysql_2"
GLUE_DB_FOR_AURORA = "sample_db"
GLUE_TABLE_FOR_AURORA = "shop"

S3_OUTPUT_LOCATION = "s3://lakehouse-sample-s3-bucket/athena/"


session = boto3.Session(
    profile_name="default",
    region_name=REGION
)

sql_query = f"""
SELECT t.*, s.name AS shop_name from "{DATASOUCE_FOR_ICEBERGE}"."{GLUE_DB_FOR_ICEBERGE}"."{GLUE_TABLE_FOR_ICEBERGE}" AS t
JOIN "{ATHENA_DATA_SOURCE_NAME}"."{GLUE_DB_FOR_AURORA}"."{GLUE_TABLE_FOR_AURORA}" AS s
ON t.shop_id = s.id;
"""

df = wr.athena.read_sql_query(sql=sql_query,
                              data_source=DATASOUCE_FOR_ICEBERGE,
                              database=GLUE_DB_FOR_ICEBERGE,
                              s3_output=S3_OUTPUT_LOCATION,
                              ctas_approach=False,
                              boto3_session=session,
)

print(df)


                                       id           timestamp  age  gender  \
0    09ddc699-14e2-469b-93ea-688f7f44abb7 2025-08-31 17:50:00   12  female   
1    f8bc1dfb-3ebc-40aa-8ad1-08ead80639cc 2025-08-31 17:50:00   15    male   
2    62b8c25b-2568-472d-91ce-f40e78484196 2025-08-31 17:50:00   19  female   
3    aeb03501-bc44-4f18-9f2b-c47b7f95b171 2025-08-31 17:50:00   13  female   
4    adb1fccc-b3cd-401c-b099-a7b77943735d 2025-08-31 17:50:01   29    male   
..                                    ...                 ...  ...     ...   
339  e32f046a-2a29-426f-b99a-29890b28fb1f 2025-08-31 17:50:59   18    male   
340  2190ce74-342b-4ef5-979b-61ed6c82cb59 2025-08-31 17:50:59   26  female   
341  1b676296-995c-4290-804f-51ef431d18e9 2025-08-31 17:50:59   12    male   
342  e9d76cc4-bc00-4449-8333-6993dd4ddef4 2025-08-31 17:50:59   26    male   
343  63a0146f-c3e9-4c55-8a40-d942c68a5f78 2025-08-31 17:50:59   27    male   

     direction  shop_id shop_name  
0           -1  0123456    